# LLM Quantization and Fine-tuning
This notebook presents how to fine-tuned a quantize model and how to quantize a fine-tuned model. From a FP16 model, both methods are roughly equivalent in terms of performance.

The fine-tuning and quantization methods employed in this notebook are recommend from a perfomance/memory/speed point of view and are good in general, but according to your specific use-case you may use alternatives not presented here.

In [ ]:
# First let's download some useful libraries - this might take several minutes

!pip install torch transformers datasets accelerate peft
!pip install torch transformers datasets optimum accelerate
!pip install git+https://github.com/casper-hansen/AutoAWQ_kernels.git -vvv
!pip install git+https://github.com/casper-hansen/AutoAWQ.git -vvv

## Fine-tuning + Quantizing
In this method, we first fine-tune the model and then quantize it.

In [ ]:
# Let's import some usefull libraries

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from awq.models import MistralAWQForCausalLM
from datasets import load_dataset
import torch
import json

### A) Fine-tuning
We use a dummy framework to LoRA fine-tune our model.

In [ ]:
# Define fine-tuning parameters

DEVICE = 'cuda:0'                                                                   # device where to load the model and perform the FT and quant on
MODEL_ID = "mistralai/Mistral-7B-v0.1"                                              # model to FT+quant, can be a hugginface id or a local path
ADAPTER_ID = "mistral-7b-v0.1_lora_boolq"                                           # name given to the fine-tuned adapter
LORA_R = 16                                                                         # lora matrices' rank
LORA_ALPHA = 8                                                                      # lora alpha scaling factor
LORA_DROPOUT = 0.05                                                                 # dropout for lora matrices when training
LORA_TARGET = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj']      # modules to apply lora FT on

In [ ]:
# Load the model

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token                                   # Needed for fine-tuning (comment out if your pre-trained model already has a padding token)

model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype=torch.bfloat16,    # Working with bfloat16 is recommended when LoRA fine-tuning
                                             device_map=DEVICE,
                                            )

In [ ]:
# Prepare model for LoRA fine-tuning

lora_config = LoraConfig(r=LORA_R,
                        lora_alpha=LORA_ALPHA,
                        lora_dropout=LORA_DROPOUT,
                        bias="none",
                        target_modules=LORA_TARGET)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Load and prepare the fine-tuning dataset
# Out of rigour, one may also use a validation set, here we skip it

def prepare_data(sample):
    prompt = f"{sample['passage']}\nQuestion: {sample['question']}?\nAnswer: {'yes' if sample['answer'] else 'no'}"
    inputs = tokenizer(prompt, padding='max_length', truncation=True, max_length=512)
    inputs.update({'labels': inputs['input_ids']})
    return inputs

dataset = load_dataset('google/boolq', split='train')
train_data = dataset.map(prepare_data).remove_columns(['answer', 'passage', 'question'])

In [ ]:
# Let's prepare the Trainer
# The training parameters should be adapted to one's model, dataset and device

training_args = TrainingArguments(
    output_dir=ADAPTER_ID,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate= 5e-5,
    num_train_epochs=1,
    evaluation_strategy="no",
    save_strategy="no",
    gradient_accumulation_steps=4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.05,
    logging_steps=1,
    bf16=True,
    #report_to='wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

In [ ]:
# Train :)

trainer.train()

In [ ]:
# Save the fine-tuned adapter
# This may be skipped if not useful

trainer.model.save_pretrained(ADAPTER_ID)
tokenizer.save_pretrained(ADAPTER_ID)

### B) Quantization
We quantize to 4 bit using AWQ.

In [ ]:
# Define quantization parameters

QUANTIZED_MODEL_ID = "mistral-7b-v0.1_lora_merged-AWQ-Q4-GS128-GEMM"               # name given to the quantized model
BITS = 4                                                                    # number of bits to quantize to, currently only 4 bits is supported
GROUP_SIZE = 128                                                            # size used for grouping in quantization algorithm, recommended 128, -1 quantizes per column
VERSION = 'GEMM'                                                            # version to quantize to ['GEMM', 'GEMV', 'GEMV_fast', 'marlin'], recommended GEMM
ZERO_POINT = True                                                           # whether to use zero-point quantization, recommend True, need False for marlin kernel

In [ ]:
# If needed you can load a fine-tuned adapter this way
# Or you can use directly the model fine-tuned just above if the notebook has not been disconnected

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype = torch.float16,       
                                             device_map=DEVICE,
                                            )
                                            
model = PeftModel(model=model, peft_config=PeftConfig.from_pretrained(ADAPTER_ID))
model.load_adapter(ADAPTER_ID, adapter_name='default')

In [ ]:
# Merge the LoRA weights to get back to the original architecture

model = model.merge_and_unload()

In [ ]:
# "Convert" the model to an AWQ version
# Note that we voluntarily specify the model architecture from awq.models rather than using AutoAWQ
# Using AutoAWQ requires using .from_pretrained which we want to circumvent here to avoid saving the full merged model
# But it would be completely possible to save the full merged model and load the weights in an AWQ architecture using .from_pretrained

quantization_config = {"zero_point": ZERO_POINT, 
                    "q_group_size": GROUP_SIZE, 
                    "w_bit": BITS, 
                    "version": VERSION, 
                    "modules_to_not_convert": [],
                    }

model = MistralAWQForCausalLM(model=model,
                            model_type=model.config.architectures[0],
                            is_quantized=False,
                            config=model.config,
                            quant_config=quantization_config,
                            processor=None)

In [ ]:
# Perform the quantization - takes 15-25mn for a 7B-parameter model on GPU

model.quantize(tokenizer, quant_config=quantization_config)

In [ ]:
# Save the quantized model

model.save_quantized(QUANTIZED_MODEL_ID)
tokenizer.save_pretrained(QUANTIZED_MODEL_ID)
with open(f"{QUANTIZED_MODEL_ID}/quant_config.json", "w") as file:
    json.dump(quantization_config, file, indent=4)

## Quantizing + Fine-tuning

In this method we first quantize the model and then fine-tune it. We use QLoRA method. Finally, we optionally propose to merge and re-quantize the whole model with AWQ.

This method is more memory efficient but slower and requires more steps. The performance is roughly the same.

In [ ]:
# Let's import some usefull libraries

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from bitsandbytes.functional import quantize_nf4, dequantize_nf4
from awq.models import MistralAWQForCausalLM
from peft.utils.other import transpose
from awq import AutoAWQForCausalLM
from safetensors import safe_open
from datasets import load_dataset
import torch
import json

### A) Quantization
We first quantize the model using QLoRA quantization.

In [ ]:
# Define quantization parameters

DEVICE = 'cuda:0'                                                           # device where to load the model and perform the FT and quant on
MODEL_ID = "mistralai/Mistral-7B-v0.1"                                      # model to FT+quant, can be a hugginface id or a local path
QUANTIZED_ADAPTER_ID = "mistral-7b-v0.1_qlora-AWQ-Q4-GS128-GEMM"            # name given to the quantized fine-tuned adapter
BITS = 4                                                                    # number of bits to quantize to, 4 or 8, we ONLY recommend 4 --> this parameter is not used in what follows
QUANT_TYPE = 'nf4'                                                          # quantization scheme ['nf4', 'fp4'], we recommend 'nf4'
DOUBLE_QUANT = False                                                        # wether to quantize the quantization constante, reduce memory but increase latency - same performance

In [ ]:
# Load the model

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token                                           # Needed for fine-tuning (comment out if your pre-trained model already has a padding token)

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4bit_compute_dtype=torch.bfloat16,      # Working with bfloat16 is recommended when LoRA fine-tuning
                                        bnb_4bit_quant_type=QUANT_TYPE,
                                        bnb_4bit_use_double_quant=DOUBLE_QUANT,
                                        )

model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype = torch.bfloat16,          # Working with bfloat16 is recommended when LoRA fine-tuning
                                             device_map=DEVICE,
                                             quantization_config=quantization_config,
                                            )

### B) Fine-tuning
We use LoRA (QLoRA) to fine-tune the model on a dummy framework.

In [ ]:
# Define fine-tuning parameters

LORA_R = 16                                                                         # lora matrices' rank
LORA_ALPHA = 8                                                                      # lora alpha scaling factor
LORA_DROPOUT = 0.05                                                                 # dropout for lora matrices when training
LORA_TARGET = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj']      # modules to apply lora FT on

In [ ]:
# Prepare model for LoRA fine-tuning

lora_config = LoraConfig(r=LORA_R,
                        lora_alpha=LORA_ALPHA,
                        lora_dropout=LORA_DROPOUT,
                        bias="none",
                        target_modules=LORA_TARGET)

model = prepare_model_for_kbit_training(model)   #! has to be applied BEFORE get_peft_model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Load and prepare the fine-tuning dataset
# Out of rigour, one may also use a validation set, here we skip it

def prepare_data(sample):
    prompt = f"{sample['passage']}\nQuestion: {sample['question']}?\nAnswer: {'yes' if sample['answer'] else 'no'}"
    inputs = tokenizer(prompt, padding='max_length', truncation=True, max_length=512)
    inputs.update({'labels': inputs['input_ids']})
    return inputs

dataset = load_dataset('google/boolq', split='train')
train_data = dataset.map(prepare_data).remove_columns(['answer', 'passage', 'question'])

In [ ]:
# Let's prepare the Trainer
# The training parameters should be adapted to one's model, dataset and device

training_args = TrainingArguments(
    output_dir=QUANTIZED_ADAPTER_ID,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate= 5e-5,
    num_train_epochs=1,
    evaluation_strategy="no",
    save_strategy="no",
    gradient_accumulation_steps=4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.05,
    logging_steps=1,
    bf16=True,
    #report_to='wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

In [ ]:
# Train :)

trainer.train()

In [ ]:
# Save the fine-tuned adapter
# This may be skipped if not useful

trainer.model.save_pretrained(QUANTIZED_ADAPTER_ID)
tokenizer.save_pretrained(QUANTIZED_ADAPTER_ID)

At this point the model is composed of the 4bit-quantized base and the FP16 LoRA-weights. One can work with this. 

However, if one wants to use the model completely quantized to 4 bits, and with its original architecture, step C which follows is required.

In [ ]:
# Note: to properly load the model with its adapter do this

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4bit_compute_dtype=torch.bfloat16,    
                                        bnb_4bit_quant_type=QUANT_TYPE,
                                        bnb_4bit_use_double_quant=DOUBLE_QUANT,
                                        )

model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype = torch.float16,      
                                             device_map=DEVICE,
                                             quantization_config=quantization_config,
                                            )
                    
model = PeftModel(model=model, peft_config=PeftConfig.from_pretrained(QUANTIZED_ADAPTER_ID))
model.load_adapter(QUANTIZED_ADAPTER_ID, adapter_name='default')

### C) Dequantize, merge and re-quantize

The final, optional, step is to make the model the exact same architecture as it originally is and entirely 4 bits. To do that we first dequantize the base model back to FP16, merge the LoRA weights within the base and finally requantize with a powerful quantization method such as AWQ.

#### 1. Dequantize base

In [ ]:
# Load the model in FP16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype = torch.float16,         
                                             device_map=DEVICE,
                                            )

In [ ]:
# Dequantize the base model
# We have to do it manually as peft enforces the re-quantization in its implemented methods, though we can still use some low-level functions from peft and bnb

for name, param in model.named_parameters():
    if any(s in name for s in ['embed', 'head', 'norm']):                                       # we keep the embeddings, head and normalization layers intact as they are always kept in higher precision
        continue
    tensor, quant_state = quantize_nf4(param.data)                                              # we quantize and dequantize other layers to reproduce the quantization error that was introduces when fine-tuning
    param.data = dequantize_nf4(tensor, quant_state=quant_state, absmax=quant_state.absmax)

#### 2. Merge LoRA weights

In [ ]:
# Load the LoRA weights in a dictionary along with the PEFT config

lora_weights = {}
with safe_open(f"{QUANTIZED_ADAPTER_ID}/adapter_model.safetensors", framework="pt", device=DEVICE) as file:
   for key in file.keys():
       lora_weights[key] = file.get_tensor(key)

with open(f"{QUANTIZED_ADAPTER_ID}/adapter_config.json", "r") as file:
    peft_config = json.load(file)

In [ ]:
for name, param in model.named_parameters():
    layer_name = name[:-7]                     
    lora_A, lora_B = None, None

    for key, val in lora_weights.items():                   # we loop on the lora_weights' keys to prevent inconsistencies in layer names, though note this name-matching part might require more work in some cases
        if f"{layer_name}.lora_A" in key:
            lora_A = val.float()
        elif f"{layer_name}.lora_B" in key:
            lora_B = val.float()
    
        if lora_A is not None and lora_B is not None:
            param.data += transpose(lora_B @ lora_A, peft_config['fan_in_fan_out']) * peft_config['lora_alpha'] / peft_config['r']
            break

#### 3. Re-quantize

In [ ]:
# Define quantization parameters

QUANTIZED_MODEL_ID = "mistral-7b-v0.1_qlora_merged-AWQ-Q4-GS128-GEMM"       # name given to the quantized model
BITS = 4                                                                    # number of bits to quantize to, currently only 4 bits is supported
GROUP_SIZE = 128                                                            # size used for grouping in quantization algorithm, recommended 128, -1 quantizes per column
VERSION = 'GEMM'                                                            # version to quantize to ['GEMM', 'GEMV', 'GEMV_fast', 'marlin'], recommended GEMM
ZERO_POINT = True                                                           # whether to use zero-point quantization, recommend True, need False for marlin kernel

In [ ]:
# "Convert" the model to an AWQ version
# Note that we voluntarily specify the model architecture from awq.models rather than using AutoAWQ
# Using AutoAWQ requires using .from_pretrained which we want to circumvent here to avoid saving the full merged model
# But it would be completely possible to save the full merged model and load the weights in an AWQ architecture using .from_pretrained

quantization_config = {"zero_point": ZERO_POINT, 
                    "q_group_size": GROUP_SIZE, 
                    "w_bit": BITS, 
                    "version": VERSION, 
                    "modules_to_not_convert": [],
                    }

model = MistralAWQForCausalLM(model=model,
                            model_type=model.config.architectures[0],
                            is_quantized=False,
                            config=model.config,
                            quant_config=quantization_config,
                            processor=None)

In [ ]:
# Perform the quantization - takes 15-25mn for a 7B-parameter model on GPU

model.quantize(tokenizer, quant_config=quantization_config)

In [ ]:
# Save the quantized model

model.save_quantized(QUANTIZED_MODEL_ID)
tokenizer.save_pretrained(QUANTIZED_MODEL_ID)
with open(f"{QUANTIZED_MODEL_ID}/quant_config.json", "w") as file:
    json.dump(quantization_config, file, indent=4)